<a href="https://colab.research.google.com/github/sathvik-ujwal/-AI-Powered-Personal-Expense-Categorization-/blob/main/expense_categorization_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from google.colab import drive
import os
drive.mount('/content/drive')
save_dir = '/content/drive/MyDrive/expense_categorization/'
os.makedirs(save_dir, exist_ok=True)
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('complex_transactions_faker1.csv')
print(df.head(10))

   transaction_id  user_id            date_time                city    amount  \
0               1     8936  2024-07-25 04:35:15               Delhi  14861.19   
1               2     3612  2025-03-06 01:47:11              Jaipur  13172.75   
2               3     5515  2024-11-12 03:03:06               Delhi   7134.89   
3               4     2046  2025-04-10 10:58:50  Thiruvananthapuram   1347.28   
4               5     1140  2025-05-01 15:41:25              Mumbai   7551.88   
5               6     5978  2025-05-03 12:12:47           Bengaluru   2469.53   
6               7     8752  2025-02-07 16:07:39              Bhopal   1892.35   
7               8      917  2025-02-23 06:08:03             Chennai  11360.52   
8               9     5156  2025-05-21 15:02:43             Chennai  18886.74   
9              10     9578  2024-09-04 18:44:05           Ahmedabad  15027.29   

  payment_method                  merchant  \
0  Mobile Wallet          Maharaj and Sons   
1  Mobile Wallet

In [5]:
df['category'].unique()

array(['Shopping', 'Food & Dining', 'Transportation', 'Travel',
       'Bills & Utilities', 'Entertainment', 'Healthcare', 'Housing',
       'Personal Care', 'Insurance', 'Education', 'Financial Obligations',
       'Miscellaneous', 'Taxes', 'Charity/Donations', 'Pets', 'Childcare'],
      dtype=object)

In [6]:
category_counts = df['category'].value_counts()
print("Category counts:\n", category_counts)

Category counts:
 category
Shopping                 20000
Food & Dining            18000
Transportation           12000
Bills & Utilities         9000
Travel                    8000
Entertainment             8000
Healthcare                5000
Housing                   5000
Personal Care             4000
Insurance                 3000
Education                 3000
Financial Obligations     3000
Miscellaneous             2000
Taxes                     2000
Charity/Donations         1000
Pets                      1000
Childcare                 1000
Name: count, dtype: int64


In [7]:
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)  # Ensure text is string
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
df['clean_desc'] = df['description'].apply(clean_text)
labels = df['category'].astype('category')
df['label'] = labels.cat.codes
label_mapping = dict(enumerate(labels.cat.categories))

In [9]:
print(label_mapping)

{0: 'Bills & Utilities', 1: 'Charity/Donations', 2: 'Childcare', 3: 'Education', 4: 'Entertainment', 5: 'Financial Obligations', 6: 'Food & Dining', 7: 'Healthcare', 8: 'Housing', 9: 'Insurance', 10: 'Miscellaneous', 11: 'Personal Care', 12: 'Pets', 13: 'Shopping', 14: 'Taxes', 15: 'Transportation', 16: 'Travel'}


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_desc'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

In [11]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

xgb = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='mlogloss')
rf = RandomForestClassifier(n_estimators=100, random_state=42)
catboost = CatBoostClassifier(iterations=100, random_state=42, verbose=0)

xgb.fit(X_train_tfidf, y_train)
rf.fit(X_train_tfidf, y_train)
catboost.fit(X_train_tfidf, y_train)

y_pred_xgb = xgb.predict(X_test_tfidf)
y_pred_rf = rf.predict(X_test_tfidf)
y_pred_catboost = catboost.predict(X_test_tfidf)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:02:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [12]:


print("=== XGBoost Classification Report ===")
print(classification_report(y_test, y_pred_xgb, target_names=labels.cat.categories))
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

print("=== Random Forest Classification Report ===")
print(classification_report(y_test, y_pred_rf, target_names=labels.cat.categories))
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("=== CatBoost Classification Report ===")
print(classification_report(y_test, y_pred_catboost, target_names=labels.cat.categories))
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)

=== XGBoost Classification Report ===
                       precision    recall  f1-score   support

    Bills & Utilities       0.97      0.79      0.87      1800
    Charity/Donations       0.98      0.79      0.87       200
            Childcare       0.97      0.71      0.82       200
            Education       0.97      0.71      0.82       600
        Entertainment       0.99      0.81      0.89      1600
Financial Obligations       1.00      0.79      0.88       600
        Food & Dining       0.63      0.89      0.74      3600
           Healthcare       0.95      0.72      0.82      1000
              Housing       0.98      0.81      0.89      1000
            Insurance       0.99      0.77      0.87       600
        Miscellaneous       0.99      0.70      0.82       400
        Personal Care       0.96      0.74      0.84       800
                 Pets       0.99      0.76      0.86       200
             Shopping       0.70      0.89      0.78      4000
                

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}
grid_xgb = GridSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    param_grid_xgb,
    cv=3,
    scoring='accuracy',
    verbose=3
)

In [20]:
grid_xgb.fit(X_train_tfidf, y_train)
print("Best parameters for XGBoost:", grid_xgb.best_params_)
print("Best cross-validation accuracy for XGBoost:", grid_xgb.best_score_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.784 total time=  20.0s
[CV 2/3] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.783 total time=  21.4s
[CV 3/3] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.782 total time=  19.9s
[CV 1/3] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.791 total time=  41.7s
[CV 2/3] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.793 total time=  42.3s
[CV 3/3] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.786 total time=  41.8s
[CV 1/3] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.804 total time= 1.5min
[CV 2/3] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.802 total time= 1.4min
[CV 3/3] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=0.796 total time= 1.4min
[CV 1/3] END learning_rate=0.01, max_depth=5, n_estimators=50;, score=0.808 to

In [22]:
import joblib
joblib.dump(grid_xgb.best_estimator_, os.path.join(save_dir, 'xgb_best_model.pkl'))

['/content/drive/MyDrive/expense_categorization/xgb_best_model.pkl']

In [24]:
import pickle
with open(os.path.join(save_dir, 'label_mapping.pkl'), 'wb') as f:
    pickle.dump(label_mapping, f)

In [25]:
def predict_category(description, model_name, model_path, vectorizer_path, label_mapping_path):
    # Load the model, vectorizer, and label mapping
    model = joblib.load(model_path)
    tfidf = joblib.load(vectorizer_path)
    with open(label_mapping_path, 'rb') as f:
        label_mapping = pickle.load(f)

    # Clean and transform the description
    clean_desc = clean_text(description)
    desc_tfidf = tfidf.transform([clean_desc])

    # Predict
    predicted_label = model.predict(desc_tfidf)[0]
    category = label_mapping[predicted_label]
    return category
